## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Araouane,ML,18.9048,-3.5265,92.41,8,12,9.73,few clouds
1,1,Saskylakh,RU,71.9167,114.0833,0.36,95,100,11.77,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,55.54,97,100,0.89,overcast clouds
3,3,Constitucion,CL,-35.3333,-72.4167,64.44,57,0,16.64,clear sky
4,4,Itacoatiara,BR,-3.1431,-58.4442,80.20,87,100,3.44,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Araouane,ML,18.9048,-3.5265,92.41,8,12,9.73,few clouds
4,4,Itacoatiara,BR,-3.1431,-58.4442,80.20,87,100,3.44,light rain
5,5,Georgetown,MY,5.4112,100.3354,83.95,87,40,2.30,scattered clouds
6,6,Caravelas,BR,-17.7125,-39.2481,83.07,66,78,8.43,broken clouds
11,11,Rikitea,PF,-23.1203,-134.9692,77.86,72,17,6.93,few clouds
12,12,Bhavnagar,IN,21.7667,72.1500,87.44,30,0,10.69,clear sky
18,18,Antalaha,MG,-14.9003,50.2788,75.20,89,51,2.91,broken clouds
20,20,Chuy,UY,-33.6971,-53.4616,76.69,44,0,8.79,clear sky
21,21,Meulaboh,ID,4.1363,96.1285,77.59,84,41,2.93,scattered clouds
26,26,Salalah,OM,17.0151,54.0924,80.69,57,0,2.30,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df = preferred_cities_df.dropna()
# preferred_cities_df.count()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Araouane,ML,92.41,few clouds,18.9048,-3.5265,
4,Itacoatiara,BR,80.20,light rain,-3.1431,-58.4442,
5,Georgetown,MY,83.95,scattered clouds,5.4112,100.3354,
6,Caravelas,BR,83.07,broken clouds,-17.7125,-39.2481,
11,Rikitea,PF,77.86,few clouds,-23.1203,-134.9692,
12,Bhavnagar,IN,87.44,clear sky,21.7667,72.1500,
18,Antalaha,MG,75.20,broken clouds,-14.9003,50.2788,
20,Chuy,UY,76.69,clear sky,-33.6971,-53.4616,
21,Meulaboh,ID,77.59,scattered clouds,4.1363,96.1285,
26,Salalah,OM,80.69,clear sky,17.0151,54.0924,


In [15]:
# 6a. Set parameters to search for hotels within 5000 meters of location.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found...skipping city.")


Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.
Hotel not found...skipping city.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.head(10)
hotel_df.info()
# clean_hotel_df = hotel_df.dropna()
# clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 685
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 217 non-null    object 
 1   Country              217 non-null    object 
 2   Max Temp             217 non-null    float64
 3   Current Description  217 non-null    object 
 4   Lat                  217 non-null    float64
 5   Lng                  217 non-null    float64
 6   Hotel Name           217 non-null    object 
dtypes: float64(3), object(4)
memory usage: 21.7+ KB


In [30]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Araouane,ML,92.41,few clouds,18.9048,-3.5265,
4,Itacoatiara,BR,80.20,light rain,-3.1431,-58.4442,BBB Rooms Centro Itacoatiara AM
5,Georgetown,MY,83.95,scattered clouds,5.4112,100.3354,Cititel Penang
6,Caravelas,BR,83.07,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
11,Rikitea,PF,77.86,few clouds,-23.1203,-134.9692,People ThankYou


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))